<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/intro_analytics_python/main/images/PA%20Banner.png" width="1000" align="center"></a>


# Probabilidad y estadística

Programa creado para mostrar ejemplos prácticos de los visto durante la clase\
v1.1

In [ ]:
import os
import platform

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
!pip install category_encoders
import category_encoders as ce

### Ensayo de probabilidad de una moneda

In [ ]:
# Ejecutar:
if os.access('partidos.csv', os.F_OK) is False:
    if platform.system() == 'Windows':
        !curl https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv > partidos.csv
    else:
        !wget partidos.csv https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/partidos.csv

In [ ]:
df = pd.read_csv("partidos.csv")
des = df.describe()
des.loc['Nan'] = df.isna().sum()
des.loc['%Nan'] = (df.isna().mean())*100
des

,home_score,away_score
count,41586.000000,41586.000000
mean,1.745756,1.187587
std,1.753780,1.405323
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000
Nan,0.000000,0.000000
%Nan,0.000000,0.000000


### Análisis de la media

In [ ]:
df.describe()

,home_score,away_score
count,41586.000000,41586.000000
mean,1.745756,1.187587
std,1.753780,1.405323
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [ ]:
df.tail()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
41581,2020-01-10,Barbados,Canada,1,4,Friendly,Irvine,United States,True
41582,2020-01-12,Kosovo,Sweden,0,1,Friendly,Doha,Qatar,True
41583,2020-01-15,Canada,Iceland,0,1,Friendly,Irvine,United States,True
41584,2020-01-19,El Salvador,Iceland,0,1,Friendly,Carson,United States,True
41585,2020-02-01,United States,Costa Rica,1,0,Friendly,Carson,United States,False


In [ ]:
# Limpiando columnas sin importancia
df_norm = pd.DataFrame(df[['date', 'home_team', 'away_team', 'neutral']])

# Añadiendo columna "win" que determina el resultado del partido
df_norm['win'] = df.apply(lambda x: 1 if x['home_score'] > x['away_score'] else 
                     0 if x['away_score'] > x['home_score'] else 0.5, axis=1)

# Si bien no dispondemos de un indicador especifico como conocer los jugadores con lo que 
# que cuenta cada equipo en cada partido, la fecha resulta ser un indicador, aunque 
# mas impreciso, para conocer dicha informacion, si 2 equipos se enfrentan entre si y si
# uno de los equipo tuvo un buen rendimiento ese mismo año, podria ser decisivo 
# para deducir con exactitud si ha sido ganador o no, por lo que nos lo vamos a quedar para el analisis 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df_norm['date'] = df_norm.apply(lambda x: x['date'][:4], axis=1) 
df_norm.loc[:, 'date'] = scaler.fit_transform(df_norm[['date']])

# Por otro lado se me ocurre hacer "data augmentation" anadiendo una columna "local" 
# cuyo valor sea de 1 o 0, Indicando si son locales o juegan de visitantes, 
# sin embargo, al hacer esto, deberia duplicar la cantidad de partidos donde existirian
# 2 registros del mismo partido cada uno desde la perspectiva del pais que se analiza
# Es esta opcion viable?


# Normalizando la columna neutral
df_norm["neutral"] = df_norm.apply(lambda x: 0 if x["neutral"] == False else 1, axis=1)


# Ejecutando el BinaryEncoder
be = ce.BinaryEncoder()
paises_df = pd.concat([df_norm['home_team'], df_norm['away_team']])
paises = pd.DataFrame(np.sort(paises_df.unique()))
be_df = be.fit_transform(paises)
paises = paises.join(be_df)

# Codificando las columnas

# Local
local_df = paises.copy()
local_df = local_df.add_prefix('local_')
df_norm = pd.merge(left=df_norm, right=local_df, left_on='home_team', right_on='local_0')

# Visitante
visitor_df = paises.copy()
visitor_df = visitor_df.add_prefix('visitor_')
df_norm = pd.merge(left=df_norm, right=visitor_df, left_on='away_team', right_on='visitor_0')

# Borrando columnas sobrantes
df_norm = df_norm.drop('home_team', axis=1)
df_norm = df_norm.drop('away_team', axis=1)
df_norm = df_norm.drop('local_0', axis=1)
df_norm = df_norm.drop('visitor_0', axis=1)

df_norm

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,date,neutral,win,local_0_0,local_0_1,local_0_2,local_0_3,local_0_4,local_0_5,local_0_6,local_0_7,local_0_8,local_0_9,visitor_0_0,visitor_0_1,visitor_0_2,visitor_0_3,visitor_0_4,visitor_0_5,visitor_0_6,visitor_0_7,visitor_0_8,visitor_0_9
0,0.000000,0,0.5,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0
1,0.013514,0,1.0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0
2,0.027027,0,1.0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0
3,0.040541,0,1.0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0
4,0.054054,0,1.0,0,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41581,0.952703,0,0.0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1
41582,0.986486,0,0.0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,1,1
41583,0.986486,0,1.0,0,0,1,1,0,1,0,0,1,1,0,1,0,0,1,1,0,0,1,1
41584,0.986486,0,0.0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0


In [ ]:
# Machine Learning con clasificador multiple
X = df_norm.drop('win', axis=1).values
y = df_norm['win'].values
in_shape = X.shape[1]
out_shape = 1

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5, criterion='entropy', max_depth=5)
clf = clf.fit(X_train, y_train)
y_hat = clf.predict(x_test)


ValueError: ignored